<a href="https://colab.research.google.com/github/DivyamNigam/Custom_Embedding_AdRecomeder_sys/blob/main/Ad_Recomendation_Mattrix_factorizaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install lightfm

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
# from lightfm import LightFM
# from lightfm.data import Dataset
# from lightfm.evaluation import precision_at_k
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
# from lightfm.evaluation import precision_at_k, recall_at_k
import numpy as np
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv("ratings_Electronics.csv")
df.columns = ['user_id', 'product_id', 'interaction', 'timestamp']

In [ ]:
df.drop(df.columns[3], axis=1, inplace=True)
df.dropna(inplace=True)
df.head()

,user_id,product_id,interaction
0,A2CX7LUOHB2NDG,0321732944,5.0
1,A2NWSAGRHCP8N5,0439886341,1.0
2,A2WNBOD3WNDNKT,0439886341,3.0
3,A1GI0U4ZRJA8WN,0439886341,1.0
4,A1QGNMC6O1VW39,0511189877,5.0


In [ ]:
df['interaction'] = np.where(df['interaction'] >= 3, 1, 0)
df.head()

,user_id,product_id,interaction
0,A2CX7LUOHB2NDG,0321732944,1
1,A2NWSAGRHCP8N5,0439886341,0
2,A2WNBOD3WNDNKT,0439886341,1
3,A1GI0U4ZRJA8WN,0439886341,0
4,A1QGNMC6O1VW39,0511189877,1


In [ ]:
user_encoder = LabelEncoder()
product_encoder = LabelEncoder()
df['user_encoded'] = user_encoder.fit_transform(df['user_id'])
df['product_encoded'] = product_encoder.fit_transform(df['product_id'])

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
len(df_train)

6259584

In [ ]:
df_train.head()

,user_id,product_id,interaction
6194284,A77RBOGEVJ0LH,B008EGUIXQ,1
157162,A3DLC0PHFF0PZE,B000067SOH,0
2708902,A2LLT3RGCRGRRE,B002MAPS6W,1
2689500,A1OLKVJAEE2ESO,B002LK2QJK,1
3325115,A263KP92EH0RY3,B003EVITCS,1


In [ ]:
a=df_train["user_id"]
a.head(5)
b=df_train["product_id"]
c=df_test['user_id']
d=df_test['product_id']

In [ ]:
# Initialize LabelEncoders
user_encoder = LabelEncoder()
product_encoder = LabelEncoder()

# Fit the encoders on the unique values of a (user_id) and b (product_id)
user_encoded = user_encoder.fit_transform(a)  # a is user_id
product_encoded = product_encoder.fit_transform(b)
user_enc_test = user_encoder.fit_transform(c)
produc_enc_test = product_encoder.fit_transform(d)


In [ ]:
X_train = np.array(list(zip(user_encoded, product_encoded)))
X_test = np.array(list(zip(user_enc_test, produc_enc_test)))

In [ ]:
X_train.size

12519168

In [ ]:
y_train = df_train['interaction']
y_test = df_test['interaction']

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(2,)),  # Input shape should be (2,) for 2 features
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=2, batch_size=32)

Epoch 1/2
195612/195612 ━━━━━━━━━━━━━━━━━━━━ 372s 2ms/step - accuracy: 0.7996 - loss: 31.1022
Epoch 2/2
195612/195612 ━━━━━━━━━━━━━━━━━━━━ 376s 2ms/step - accuracy: 0.8264 - loss: 0.4617


In [ ]:
# Make predictions on the test data
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
print(f"RMSE: {rmse:.4f}")

48904/48904 ━━━━━━━━━━━━━━━━━━━━ 69s 1ms/step
RMSE: 0.3787
